In [1]:
import requests
import bs4
from bs4 import BeautifulSoup
import pandas as pd
import sqlite3

In [2]:
page = requests.get('https://www.cricbuzz.com/cricket-series/2697/icc-cricket-world-cup-2019/matches')

In [3]:
cricPage = BeautifulSoup(page.content, 'html.parser')

In [4]:
resultTable = cricPage.find_all(class_ = 'cb-text-complete')

In [16]:
resultList = []

for match in resultTable:
    row = list(match.children)[0]
        
    if 'won' in row:
        resultDict = {}
        textSplit = row.split("won")
        resultDict['Result'] = textSplit[0].rstrip()
        resultList.append(resultDict)
        
    elif 'abandoned' or 'No result' in row:
        resultDict = {}
        resultDict['Result'] = 'NR'
        resultList.append(resultDict)

In [17]:
df = pd.DataFrame(resultList)

In [20]:
updated_results = df[10:].reset_index(drop =True)

In [22]:
conn = sqlite3.connect('../database/results.sqlite')

In [23]:
updated_results.to_sql("results", conn, if_exists= 'replace', index = False)

In [24]:
conn.close()